The code below simulates hand odds in a hand crafted version of Goofspiel.

The deck is made up of 7 suits:
* Red
* Orange
* Yellow
* Green
* Blue
* Indigo
* Violet

Each suit has 10 ranks, 1 through 10. Each rank has two copies. Finally, there are 7 black cards all ranked -10. This creates a total of 147 cards.

The amount of cards and suits varies depending on the players.

Without going into detail on how the game works, each player is attempting to build different hands from a limited selection of auction cards.

Using the Monte Carlo method we can determine the odds of various hands occuring based on the restrictions set in place.

Those main 6 hands include:
* Pairs
* Runs
* Flushes
* Rainbows
* Half Rainbows
* Blackholes

For this analysis we count an occurance as true if the necessary cards appear anywhere within the available auction cards. Position is not a concern. Neither is the probability of winning that card on demand.

The generated table was created after simulating each hand for each player amount 1 million times. In the case that the odds were less than 0.01% I have rounded them to 0. This is mainly due to the assumption that the odds of the hand being a possibility were so astronomical that it would not be worth considering during normal play of the game.

In [212]:
# Importing necessary libraries
import random
import copy
from collections import Counter
from functools import reduce
import pandas as pd
import numpy as np

In [213]:
# Adjusts number of active suits and total available auction cards based on the number of players
def player_adjustment(players):
  if players == 2:
    total_suits = 2
    auction_cards = 10
    return (total_suits, auction_cards)
  elif players == 3:
    total_suits = 3
    auction_cards = 10
    return (total_suits, auction_cards)
  elif players == 4:
    total_suits = 4
    auction_cards = 20
    return (total_suits, auction_cards)
  elif players == 5:
    total_suits = 5
    auction_cards = 20
    return (total_suits, auction_cards)
  elif players == 6:
    total_suits = 6
    auction_cards = 30
    return (total_suits, auction_cards)
  elif players == 7:
    total_suits = 7
    auction_cards = 30
    return (total_suits, auction_cards)

# Builds an organized deck based on number of players
def build_deck(players):
  organized_deck = []
  adjustment = player_adjustment(players)[0]
  ranks = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'T']
  suits = ['R', 'O', 'Y', 'G', 'B', 'I', 'V']
  black_cards = ['NJ', 'NJ', 'NJ', 'NJ', 'NJ', 'NJ', 'NJ']
  for rank in ranks:
    for suit in suits[0:adjustment]:
      organized_deck.append(rank+suit)
  for card in black_cards[0:adjustment]:
    organized_deck.append(card)
  return organized_deck

# Returns the rank of all auction cards
def get_ranks(deck, players):
  auction_cards = player_adjustment(players)[1]
  cards = deck[0:auction_cards]
  list_of_ranks = []
  for i in range(len(cards)):
    list_of_ranks.append(cards[i][0])
  return list_of_ranks

# Returns the suit of all auction cards
def get_suits(deck, players):
  auction_cards = player_adjustment(players)[1]
  cards = deck[0:auction_cards]
  list_of_ranks = []
  for i in range(len(cards)):
    list_of_ranks.append(cards[i][1])
  return list_of_ranks

# Returns the number of pairs in the available auction cards
def pair(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//2 for i in Counter(check_list).values())
  return results

# Returns the number of three of a kind in the available auction cards
def three_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//3 for i in Counter(check_list).values())
  return results

# Returns the number of four of a kind in the available auction cards
def four_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//4 for i in Counter(check_list).values())
  return results

# Returns the number of five of a kind in the available auction cards
def five_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//5 for i in Counter(check_list).values())
  return results

# Returns the number of six of a kind in the available auction cards
def six_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//5 for i in Counter(check_list).values())
  return results

# Returns the number of seven of a kind in the available auction cards
def seven_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//5 for i in Counter(check_list).values())
  return results

# Returns the number off flushes in the available auction cards
def flush(deck, players):
  check_list = get_suits(deck, players)
  results = sum(i//3 for i in Counter(check_list).values())
  return results

# Checks whether a black hole occurs
def black_hole(deck, players):
  cards = get_ranks(deck, players)
  count_of_cards = Counter(cards)
  black_cards = count_of_cards['N']
  if black_cards == players:
    return True

# Checks whether a rainbow occurs
def rainbow(deck, players):
  suit_counts = get_suit_counts(deck, players)
  R = suit_counts[0]
  O = suit_counts[1]
  Y = suit_counts[2]
  G = suit_counts[3]
  B = suit_counts[4]
  I = suit_counts[5]
  V = suit_counts[6]
  if R > 0 and O > 0 and Y > 0 and G > 0 and B > 0 and I > 0 and V > 0:
    return True

def get_suit_counts(deck, players):
  cards = get_suits(deck, players)
  count_of_cards = Counter(cards)
  R = count_of_cards['R']
  O = count_of_cards['O']
  Y = count_of_cards['Y']
  G = count_of_cards['G']
  B = count_of_cards['B']
  I = count_of_cards['I']
  V = count_of_cards['V']
  suit_counts = [R, O, Y, G, B, I, V]
  return suit_counts

# Checks whether a half rainbow occurs
def half_rainbow(deck, players):
  suit_counts = get_suit_counts(deck, players)
  r = suit_counts[0]/(suit_counts[0]+1)
  o = suit_counts[1]/(suit_counts[1]+1)
  y = suit_counts[2]/(suit_counts[2]+1)
  g = suit_counts[3]/(suit_counts[3]+1)
  b = suit_counts[4]/(suit_counts[4]+1)
  i = suit_counts[5]/(suit_counts[5]+1)
  v = suit_counts[6]/(suit_counts[6]+1)
  total = r + o + y + g + b + i + v
  if total >= 4 and total < 7:
    return True

# Function that converts the string ranks to integers
def ranks_to_integers(rank):
  match rank:
    case '1': return 1
    case '2': return 2
    case '3': return 3
    case '4': return 4
    case '5': return 5
    case '6': return 6
    case '7': return 7
    case '8': return 8
    case '9': return 9
    case 'T': return 10
    case 'N': return -10

# Checks whether the values in a list are sequential
def is_sequential(list_of_values):
  return np.sum (np.diff(sorted(list_of_values))) == len(list_of_values) - 1

# Returns the number of runs present in the available auction cards
def run(deck, players, run_length):
  cards = get_ranks(deck, players)
  converted_rank = []
  # Creating set of ranks converted to integers
  for card in cards:
    converted = ranks_to_integers(card)
    converted_rank.append(converted)
  converted_rank_set = list(set(converted_rank))
  list_of_sequences = []
  # Creating list of card sequences to check
  if run_length <= len(converted_rank_set):
    for i in range(len(converted_rank_set) - (run_length - 1)):
      sequences_to_check = converted_rank_set[i:i+run_length]
      list_of_sequences.append(sequences_to_check)
  run_counter = 0
  # Checking whether each sequence is sequential
  for sequence in list_of_sequences:
    if is_sequential(sequence):
      run_counter += 1
  return run_counter

# Returns the number of flush_runs present in the available auction cards
def flush_run(deck, players, run_length):
  # Counting the number of occurences of each suit
  cards = deck[0:player_adjustment(players)[1]]
  check_list = get_suits(deck, players)
  count_of_cards = Counter(check_list)
  R = count_of_cards['R']
  O = count_of_cards['O']
  Y = count_of_cards['Y']
  G = count_of_cards['G']
  B = count_of_cards['B']
  I = count_of_cards['I']
  V = count_of_cards['V']
  count_of_suits = [('R', R), ('O', O), ('Y', Y), ('G', G), ('B', B), ('I', I), ('V', V)]
  # Creating list of suits that have flush potential
  valid_suits = []
  for i in range(len(count_of_suits)):
    if count_of_suits[i][1] >= 3: valid_suits.append(count_of_suits[i][0])
  # Creating list of ranks partitioned by suit
  r_ranks = []
  o_ranks = []
  y_ranks = []
  g_ranks = []
  b_ranks = []
  i_ranks = []
  v_ranks = []
  for i in range(len(cards)):
    if cards[i][1] in valid_suits:
      match cards[i][1]:
        case 'R': r_ranks.append(cards[i][0])
        case 'O': o_ranks.append(cards[i][0])
        case 'Y': o_ranks.append(cards[i][0])
        case 'G': o_ranks.append(cards[i][0])
        case 'B': o_ranks.append(cards[i][0])
        case 'I': o_ranks.append(cards[i][0])
        case 'V': o_ranks.append(cards[i][0])
  suit_ranks = [r_ranks, o_ranks, y_ranks, g_ranks, b_ranks, i_ranks, v_ranks]
  # Creating list of ranks converted to integers
  converted_suit_rank = []
  for i in suit_ranks:
    if len(i) > 0:
      converted_rank = []
      for j in i:
        converted = ranks_to_integers(j)
        converted_rank.append(converted)
      converted_suit_rank.append(converted_rank)
  # Removing duplicates and sorting the converted ranks
  converted_suit_rank_set = []
  for i in converted_suit_rank:
    list_to_set = list(set(i))
    converted_suit_rank_set.append(list_to_set)
  # Creating list of sequences to check
  list_of_sequences = []
  for i in converted_suit_rank_set:
    if run_length <= len(i):
      for j in range(len(i) - (run_length - 1)):
        sequences_to_check = i[j:j+run_length]
        list_of_sequences.append(sequences_to_check)
  run_counter = 0
  # Checking whether each sequence is sequential
  for sequence in list_of_sequences:
    if is_sequential(sequence):
      run_counter += 1
  return run_counter

# Returns list of counts of each run type
def run_counter(deck, players):
  count_of_runs = []
  for i in range(3,11):
    results = run(deck, players, i)
    count_of_runs.append(results)
  return count_of_runs

# Returns list of counts for each flush run type
def flush_run_counter(deck, players):
  count_of_flushes = []
  for i in range(3,11):
    results = flush_run(deck, players, i)
    count_of_flushes.append(results)
  return count_of_flushes


# Randomly shuffles the created deck and checks to see if given conditions are met
# Returns dataframe of the percentage that a positive result occurred
# The more iterations the more auccurate the results
def MonteCarlo(iterations, players):
  # build the deck
  organized_deck = build_deck(players)

  # Initiate the hand counts
  final_no_pair_count = 0
  final_pair_count = 0
  final_two_pair_count = 0
  final_three_pair_count = 0
  final_four_pair_count = 0
  final_five_pair_count = 0
  final_six_pair_count = 0
  final_seven_pair_count = 0
  final_eight_pair_count = 0
  final_nine_pair_count = 0
  final_ten_pair_count = 0
  final_eleven_pair_count = 0
  final_twelve_pair_count = 0
  final_thirteen_pair_count = 0
  final_fourteen_pair_count = 0
  final_fifteen_pair_count = 0
  final_three_kind_count = 0
  final_four_kind_count = 0
  final_five_kind_count = 0
  final_six_kind_count = 0
  final_seven_kind_count = 0
  final_flush_count = 0
  final_run_3_count = 0
  final_run_4_count = 0
  final_run_5_count = 0
  final_run_6_count = 0
  final_run_7_count = 0
  final_run_8_count = 0
  final_run_9_count = 0
  final_run_10_count = 0
  final_flush_run_3_count = 0
  final_flush_run_4_count = 0
  final_flush_run_5_count = 0
  final_flush_run_6_count = 0
  final_flush_run_7_count = 0
  final_flush_run_8_count = 0
  final_flush_run_9_count = 0
  final_flush_run_10_count = 0
  black_hole_counter = 0
  rainbow_counter = 0
  half_rainbow_counter = 0

  # Beginning the loop
  for i in range(iterations):

    # Copy the deck and shuffle it
    deck = copy.deepcopy(organized_deck)
    random.shuffle(deck)

    # Counting pairs
    pair_count = pair(deck, players)
    match pair_count:
      case 0: final_no_pair_count +=1
      case 1: final_pair_count +=1
      case 2: final_two_pair_count +=1
      case 3: final_three_pair_count +=1
      case 4: final_four_pair_count +=1
      case 5: final_five_pair_count +=1
      case 6: final_six_pair_count +=1
      case 7: final_seven_pair_count +=1
      case 8: final_eight_pair_count +=1
      case 9: final_nine_pair_count +=1
      case 10: final_ten_pair_count +=1
      case 11: final_eleven_pair_count +=1
      case 12: final_twelve_pair_count +=1
      case 13: final_thirteen_pair_count +=1
      case 14: final_fourteen_pair_count +=1
      case 15: final_fifteen_pair_count +=1

    # Counting x of a kind
    three_kind_count = three_kind(deck, players)
    final_three_kind_count += three_kind_count
    four_kind_count = four_kind(deck, players)
    final_four_kind_count += four_kind_count
    five_kind_count = five_kind(deck, players)
    final_five_kind_count += five_kind_count
    six_kind_count = six_kind(deck, players)
    final_six_kind_count += six_kind_count
    seven_kind_count = seven_kind(deck, players)
    final_seven_kind_count += seven_kind_count
    flush_count = flush(deck, players)
    final_flush_count += flush_count

    # Counting runs
    run_count = run_counter(deck, players) # Returns list
    run_3 = run_count[0]
    final_run_3_count += run_3
    run_4 = run_count[1]
    final_run_4_count += run_4
    run_5 = run_count[2]
    final_run_5_count += run_5
    run_6 = run_count[3]
    final_run_6_count += run_6
    run_7 = run_count[4]
    final_run_7_count += run_7
    run_8 = run_count[5]
    final_run_8_count += run_8
    run_9 = run_count[6]
    final_run_9_count += run_9
    run_10 = run_count[7]
    final_run_10_count += run_10

    # Counting flush runs
    flush_run_count = flush_run_counter(deck, players) # Returns list
    flush_run_3 = flush_run_count[0]
    final_flush_run_3_count += flush_run_3
    flush_run_4 = flush_run_count[1]
    final_flush_run_4_count += flush_run_4
    flush_run_5 = flush_run_count[2]
    final_flush_run_5_count += flush_run_5
    flush_run_6 = flush_run_count[3]
    final_flush_run_6_count += flush_run_6
    flush_run_7 = flush_run_count[4]
    final_flush_run_7_count += flush_run_7
    flush_run_8 = flush_run_count[5]
    final_flush_run_8_count += flush_run_8
    flush_run_9 = flush_run_count[6]
    final_flush_run_9_count += flush_run_9
    flush_run_10 = flush_run_count[7]
    final_flush_run_10_count += flush_run_10

    # Counting black holes and rainbows
    if black_hole(deck, players): black_hole_counter += 1
    if rainbow(deck, players): rainbow_counter += 1
    if half_rainbow(deck, players): half_rainbow_counter += 1
  final_counts = [final_no_pair_count, final_pair_count
                  ,final_two_pair_count, final_three_pair_count
                  ,final_four_pair_count, final_five_pair_count
                  ,final_six_pair_count, final_seven_pair_count
                  ,final_eight_pair_count, final_nine_pair_count
                  ,final_ten_pair_count, final_eleven_pair_count
                  ,final_twelve_pair_count, final_thirteen_pair_count
                  ,final_fourteen_pair_count, final_fifteen_pair_count
                  ,final_three_kind_count, final_four_kind_count
                  ,final_five_kind_count, final_six_kind_count
                  ,final_seven_kind_count, final_run_3_count
                  ,final_run_4_count, final_run_5_count, final_run_6_count
                  ,final_run_6_count, final_run_7_count, final_run_8_count
                  ,final_run_9_count, final_run_10_count
                  ,final_flush_run_3_count, final_flush_run_4_count
                  ,final_flush_run_5_count, final_flush_run_6_count
                  ,final_flush_run_8_count, final_flush_run_9_count
                  ,final_flush_run_10_count, black_hole_counter
                  ,rainbow_counter, half_rainbow_counter]
  final_aggregations = []
  for i in final_counts:
    percentage = round(((i/iterations)*100), 2)
    if percentage > 100:
      percentage = 100.0
    final_aggregations.append(percentage)
    data = {
    'Hand_Type': ['No_Pair', 'Pair','Two_Pair','Three_Pair','Four_Pair'
                  ,'Five_Pair','Six_Pair','Seven_Pair','Eight_Pair','Nine_Pair'
                  ,'Ten_Pair','Eleven_Pair','Twelve_Pair','Thirteen_Pair'
                  ,'Fourteen_Pair','Fifteen_Pair', 'Three_Kind', 'Four_Kind'
                  ,'Five_Kind', 'Six_Kind','Seven_Kind', 'Run_of_3', 'Run_of_4'
                  ,'Run_of_5', 'Run_of_6','Run_of_7', 'Run_of_8', 'Run_of_9'
                  ,'Run_of_10', 'Flush_Run_3', 'Flush_Run_4', 'Flush_Run_5'
                  ,'Flush_Run_6','Flush_Run_7', 'Flush_Run_8'
                  ,'Flush_Run_9', 'Flush_Run_10','Black_Hole'
                  , 'Rainbow', 'Half_Rainbow']
    ,'Percentage': final_aggregations
        }
  df = pd.DataFrame(data)
  return df

In [214]:
data_frames = []
for i in range(2,8):
  iterations = 1000000
  df = MonteCarlo(iterations, i)
  players= str(i)
  players_column = players+'_Players'
  df.rename(columns={'Hand_Type': 'Hand_Type', 'Percentage': players_column},inplace = True)
  data_frames.append(df)

df_all = reduce(lambda  left,right: pd.merge(left,right,on=['Hand_Type'],how='outer'), data_frames)
df_all

,Hand_Type,2_Players,3_Players,4_Players,5_Players,6_Players,7_Players
0,No_Pair,1.76,0.71,0.00,0.00,0.00,0.00
1,Pair,19.57,13.63,0.00,0.00,0.00,0.00
2,Two_Pair,45.75,44.27,0.00,0.00,0.00,0.00
3,Three_Pair,28.56,34.94,0.00,0.00,0.00,0.00
4,Four_Pair,4.29,6.34,0.00,0.00,0.00,0.00
5,Five_Pair,0.07,0.12,1.05,1.09,0.00,0.00
6,Six_Pair,0.00,0.00,16.07,16.23,0.00,0.00
7,Seven_Pair,0.00,0.00,45.14,45.12,0.00,0.00
8,Eight_Pair,0.00,0.00,32.26,32.17,0.00,0.00
9,Nine_Pair,0.00,0.00,5.37,5.30,0.00,0.00


In [218]:
df_all.to_excel('/content/drive/MyDrive/Goofspiel/Hand Odds.xlsx', index=False)